In [ ]:
#一. 什么是tfdbg?
#经过之前的学习，我们发现构建一个TensorFlow 项目需要经历两个步骤：
#1. 设计图
#2. 使用 Session.run() 运行图
#如果在第二阶段出现了错误和 bug，我们很难进行调试，因为我们无法在运行时断点进入图中（封闭的）。所以，一个专用的运行环境调试器（debugger）是目前 TensorFlow 用户所急需的工具。
#因此，伴随着Tensorflow r1.0的发布，tfdbg这一调试器也随之发布


#二. tensorflow自带性能分析工具
#参见：http://stackoverflow.com/questions/37751739/tensorflow-code-optimization-strategy 

In [ ]:
# coding=utf-8  
''''' 
测试Tensorflow的性能分析工具; 
该工具也可以检测变量的位置，， 
 
参考网址：http://stackoverflow.com/questions/37751739/tensorflow-code-optimization-strategy 
 
Created on Mar 30, 2017 
@author: colinliang 
'''  
from __future__ import absolute_import  
from __future__ import division  
from __future__ import print_function  
import tensorflow as tf  
from tensorflow.python.client import timeline  
if __name__ == '__main__':  
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)  
    run_metadata = tf.RunMetadata()  
    sess = tf.Session()  
    print(u'CUPTI的so文件位置变了, 直接运行本程序会产生exception！！！ http://blog.csdn.net/rtygbwwwerr/article/details/51605835')  
    print(u'解决方法：我是拷贝了一下/usr/local/cuda/extras/CUPTI/lib64对应3个的文件到 /usr/local/cuda/lib64 中')  
    from tensorflow.python.ops import partitioned_variables  
    partitioner = partitioned_variables.variable_axis_size_partitioner(max_shard_bytes=512)  # 注意这里是分割的字节数，而不是float的数量  
    dim = 2  
      
#     with tf.device('/cpu:0'):  
#         embedding_var=tf.get_variable('embedding_var', shape=[600,dim],partitioner=partitioner, dtype=tf.float32)  
    embedding_var = tf.get_variable('embedding_var', shape=[200, dim], partitioner=partitioner, dtype=tf.float32)   
      
    w = tf.get_variable('w', shape=[dim, 10], dtype=tf.float32)  
      
#     tf.PartitionedVariable  # 如果进行了partition，变量类型不是tf.Variable，而是，由一组tf.Variable组成的tf.PartitionedVariable  
    sess.run(tf.global_variables_initializer())  
    print('embedding_var: %s' % embedding_var)  
#     print('device of embedding_var: %s'% embedding_var.device)  #对于单个变量，可以这么打印device，但只针对显式指定device的变量有效。。。  
  
    r = tf.nn.embedding_lookup(embedding_var, [0])  
    r = tf.matmul(r, w)  
    print(r)  
    sess.run(r, options=run_options, run_metadata=run_metadata)  
          
#     print(run_metadata)  
    tl = timeline.Timeline(run_metadata.step_stats)  
    ctf = tl.generate_chrome_trace_format()  
    tracing_log='D:\workflow\code\tensorflow\timeline.json'  
    with open(tracing_log, 'w') as f:  
        f.write(ctf)  
    print('DONE')  
    print('在chrome中打开：   chrome://tracing  ， 再load %s 即可查看运行日志'%tracing_log)  
    exit(0)  

CUPTI的so文件位置变了, 直接运行本程序会产生exception！！！ http://blog.csdn.net/rtygbwwwerr/article/details/51605835
解决方法：我是拷贝了一下/usr/local/cuda/extras/CUPTI/lib64对应3个的文件到 /usr/local/cuda/lib64 中
